In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
import pingouin as pg



params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
###get inactivation opto

id_date = { 
"004072" : ["20230717"],
"004075" : ["20230717","20230825"],
"004073" : ["20230808","20230811"],
"004109" : ["20230808","20230811"],
"004110" : ["20230808","20230815"],
"004111" : ["20230815"],
"004112" : ["20230811","20230815"]
}
sess =1
pre = 10
post = 20
df_list = []
labels = list(range(1,5))
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        binned_slope = np.array(pd.qcut(slopes,q = 4,labels = labels))
        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"baseline":binned_baseline[j],"slope_q":binned_slope[j],"slope":slopes[j]}
                df_list.append(new_dict)

vgat_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###get inactivation opto for V1

##v1
id_date = { 
"004253": ["20240520"],
"004254": ["20240520"],
"004255": ["20240520"],
"004256": ["20240520"],
"004257": ["20240520"],
"004258": ["20240520"],
}
sess =1
pre = 10
post = 20
df_list = []
labels = list(range(1,5))
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        binned_slope = np.array(pd.qcut(slopes,q = 4,labels = labels))
        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"baseline":binned_baseline[j],"slope_q":binned_slope[j],"slope":slopes[j]}
                df_list.append(new_dict)

v1_df = pd.DataFrame.from_dict(df_list)

In [ ]:
ax = sns.relplot(data = vgat_df, x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,kind = "line",col =  "ID",row ="date")


In [ ]:
ax = sns.lineplot(data = vgat_df, x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1)


In [ ]:
###get new mcherry data
id_date = { 
 "004259" : ["20240513"],
 "004260" : ["20240513"],
"004261" : ["20240513"],
"004262" : ["20240513"],
"004264" : ["20240513"],
}
sess =1
pre = 10
post = 20
df_list = []
labels = list(range(1,5))
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        binned_slope = np.array(pd.qcut(slopes,q = 4,labels = labels))
        for j,trial in enumerate(pupil_snips):
            baseline = np.nanmean(trial[160:200])
            trial = trial - baseline
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i], "ID" : id, "date" : date,"sess":sess,"baseline":binned_baseline[j],"slope_q":binned_slope[j],"slope":slopes[j]}
                df_list.append(new_dict)

mcherry_df = pd.DataFrame.from_dict(df_list)

In [ ]:
mcherry_df

In [ ]:
##mcherry control data

id_date = { 
"000033" : [["20221018",2],["20221018",3]],
"000034" : [["20221014",2],["20221019",3]],
"000035" : [["20221014",2],["20221014",3]],
"000036" : [["20221014",2],["20221019",3]],

}



labels = list(range(1,5))
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date,sess in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        laser_times = loadmat(matpath)["data"]["response"]["laser_time"]
        df = pd.read_csv(pupil_path)
        time = df['Time'].to_numpy()
        areas = df['Pupil Area'].to_numpy()
        trial_matrix,peri_times = trial_align(laser_times,time,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in trial_matrix:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        binned_slope = np.array(pd.qcut(slopes,q = 4,labels = labels))


        for j,trial in enumerate(trial_matrix):
            for i,time in enumerate(peri_times):
                baseline = np.nanmean(trial[160:200])
                trial = trial - baseline
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"ID":id,"date":date,"sess":sess,"baseline":binned_baseline[j],"slope_q":binned_slope[j],"slope":slopes[j]}
                df_list.append(new_dict)

mcherry_df1 = pd.DataFrame.from_dict(df_list)


In [ ]:
mcherry_df = pd.concat([mcherry_df,mcherry_df1])

In [ ]:
###dff responses over time
def label_trial(row):
    if row['slope']<-0.02:
      return 'Constricting'
    if row['slope']>0.02:
      return 'Dilating'
    else:
      return "Stable"

vgat_df['slope_bin'] = vgat_df.apply(label_trial, axis=1)
mcherry_df['slope_bin'] = mcherry_df.apply(label_trial, axis=1)

In [ ]:
plt.figure(figsize = (1.2,1.2))
ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Dilating'"), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
ax = sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Dilating'"), x = "Time from Laser (s)", y = "Z-Scored Pupil Area",errorbar = "se",linewidth = 1,color = "grey",legend = None)
format_ax(ax,[-1,6],[-0.5,1],1,1)
ax.axvspan(0,5,alpha = 0.2)



In [ ]:
ax = sns.lineplot(data = vgat_df.query("slope_bin == 'Dilating'"), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = True,hue = "ID")


In [ ]:
plt.figure(figsize = (2.2,2.2))
ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Stable'"), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
ax = sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Stable'"), x = "Time from Laser (s)", y = "Z-Scored Pupil Area",errorbar = "se",linewidth = 1,color = "grey",legend = None)
format_ax(ax,[-2,6],[-0.5,1.5],1,1)
ax.axvspan(0,5,alpha = 0.2)



In [ ]:
plt.figure(figsize = (2.2,2.2))
ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Constricting'"), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
ax = sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)","slope_bin"]).mean().reset_index().query("slope_bin == 'Constricting'"), x = "Time from Laser (s)", y = "Z-Scored Pupil Area",errorbar = "se",linewidth = 1,color = "grey",legend = None)
#format_ax(ax,[-2,6],[-0.5,1.5],1,1)
ax.axvspan(0,5,alpha = 0.2)



In [ ]:
ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)","slope_bin"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,hue = "slope_bin",legend = True)
format_ax(ax,[-1,6],[-1,2],1,1)


In [ ]:
###plot trials split by baseline pupil quartile
plt.figure(figsize = (1.2,1.2))
ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)","baseline"]).mean().reset_index().query("baseline == [3]"), x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
ax = sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)","baseline"]).mean().reset_index().query("baseline == [3]"), x = "Time from Laser (s)", y = "Z-Scored Pupil Area",errorbar = "se",linewidth = 1,color = "grey",legend = None)

ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-6,6],[-1,2],1,1)
ax.set_xlabel("Time from laser (s)")
ax.set_ylabel("Pupil size (z-scr)")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_q4.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
plt.figure(figsize = (1.6,1.6))

ax = sns.lineplot(data = vgat_df.groupby(["ID","type","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1)
ax = sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "grey",linewidth = 1,alpha = 0.5)

#sns.lineplot(data = mcherry_df.groupby(["ID","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "Z-Scored Pupil Area",errorbar = "se",ax = ax,color = "grey",linewidth = 1,alpha = 0.5)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,8],[-0.4,0.3],2,0.1)
ax.set_xlabel("Time from laser (s)")
ax.set_ylabel("Pupil size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
ax = sns.relplot(data = mcherry_df1, x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1,kind = "line",col =  "ID",col_wrap =3)
for ax in ax.axes:
    ax.axvspan(0,5,alpha = 0.2)
    format_ax(ax,[-3,10],[-0.6,0.8],2,0.2)
   

In [ ]:
mcherry_df1

In [ ]:
###v1 control opto stim
plt.figure(figsize = (1.9,1.6))
ax = sns.lineplot(data = v1_df.groupby(["ID","type","Time from Laser (s)"]).mean().reset_index(), x = "Time from Laser (s)", y = "value",errorbar = "se",color = "dodgerblue",linewidth = 1)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,8],[-0.4,0.3],2,0.1)
ax.set_xlabel("Time from laser (s)")
ax.set_ylabel("Pupil size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/v1_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###get vgat bar

id_date = { 
"004072" : ["20230717"],
"004075" : ["20230717","20230825"],
"004073" : ["20230808","20230811"],
"004109" : ["20230808","20230811"],
"004110" : ["20230808","20230815"],
"004111" : ["20230815"],
"004112" : ["20230811","20230815"]
}

##v1
# id_date = { 
# "004253": ["20240520"],
# "004254": ["20240520"],
# "004255": ["20240520"],
# "004256": ["20240520"],
# "004257": ["20240520"],
# "004258": ["20240520"],

# }


sess =1
pre = 10
post = 10
df_list = []
delta_list = []

for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

           
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)

            new_dict = {"Delta" : post_laser - pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j}
            delta_list.append(new_dict)





bar_df = pd.DataFrame.from_dict(df_list)
delta_df = pd.DataFrame.from_dict(delta_list)

In [ ]:
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-0.25,0.5,0.25))
#g.set(ylim=(-0.25, 0.25))
g.set_ylabel("Pupil Size\n(z-scr)")
g.xaxis.label.set_visible(False)

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_pupil_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (0.5,1.2))
g = sns.barplot(data = delta_df.groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.stripplot(data = delta_df.groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.2)

g.set_yticks(np.arange(-0.5,0.75,0.25))
#g.set(ylim=(-0.5, 0.5))
g.set_ylabel("∆ Pupil size (z-scr)")

g.set_xlabel("ChR2")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = delta_df.groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
###get new mcherry bar

id_date = { 
"004259": ["20240513"],
"004260": ["20240513"],
 "004261": ["20240513"],
 "004262": ["20240513"],
"004263": ["20240513"],
"004264": ["20240513"],

}
sess =1
pre = 10
post = 10
df_list = []
delta_list = []

for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

           
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)

            new_dict = {"Delta" : post_laser - pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j}
            delta_list.append(new_dict)





bar_df = pd.DataFrame.from_dict(df_list)
mcherry_delta_df = pd.DataFrame.from_dict(delta_list)

In [ ]:
###mcherry bar
###mcherry control data

id_date = { 
"000033" : [["20221018",2],["20221018",3]],
"000034" : [["20221014",2],["20221019",3]],
"000035" : [["20221014",2],["20221014",3]],
"000036" : [["20221014",2],["20221019",3]],
}

pre = 10
post = 10
df_list = []
delta_list = []
for id,dates in id_date.items():
    for date,sess in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        laser_times = loadmat(matpath)["data"]["response"]["laser_time"]
        df = pd.read_csv(pupil_path)
        time = df['Time'].to_numpy()
        areas = df['Pupil Area'].to_numpy()
        trial_matrix,peri_times = trial_align(laser_times,time,areas,pre = pre, post = post)

        for j,trial in enumerate(trial_matrix):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"mCherry","Date":date,"trial":j}
            df_list.append(new_dict)

            new_dict = {"Delta" : post_laser - pre_laser,"ID" : id,'Condition':"mCherry","Date":date,"trial":j}
            delta_list.append(new_dict)

bar_df = pd.DataFrame.from_dict(df_list)
mcherry_delta_df1 = pd.DataFrame.from_dict(delta_list)



In [ ]:
mcherry_delta_df = pd.concat([mcherry_delta_df,mcherry_delta_df1])

In [ ]:
###stats
stat_df = mcherry_delta_df.groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0,0,0,0,0]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.5,0.9))
g = sns.barplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black",order = ["Pre-Laser","Post-Laser"])
sns.lineplot(data = bar_df.groupby(["ID",'laser']).mean().reset_index(), x = "laser", y = "value",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(-0.25,0.5,0.25))
g.set(ylim=(-0.25, 0.25))
g.set_ylabel("Pupil size (z-scr)")
g.xaxis.label.set_visible(False)

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgatmcherry_pupil_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
plt.figure(figsize = (0.5,1.2))
g = sns.barplot(data = mcherry_delta_df.groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black")
sns.stripplot(data = mcherry_delta_df.groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.2)

g.set_yticks(np.arange(-0.5,0.75,0.25))
g.set(ylim=(-0.5, 0.5))
g.yaxis.label.set_visible(False)
g.axes.yaxis.set_ticklabels([])

g.set_xlabel("mCherry")


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###get v1 bar

##v1
id_date = { 
"004253": ["20240520"],
"004254": ["20240520"],
"004255": ["20240520"],
"004256": ["20240520"],
"004257": ["20240520"],
"004258": ["20240520"],

}


sess =1
pre = 10
post = 10
df_list = []
delta_list = []

for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

           
            new_dict = {"value" : post_laser,"ID" : id,"laser": "Post-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)
            new_dict = {"value" : pre_laser,"ID" : id,"laser": "Pre-Laser",'Condition':"VGAT","Date":date,"trial":j}
            df_list.append(new_dict)

            new_dict = {"Delta" : post_laser - pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j}
            delta_list.append(new_dict)





bar_df = pd.DataFrame.from_dict(df_list)
delta_df = pd.DataFrame.from_dict(delta_list)

In [ ]:
###stats
stat_df = delta_df.groupby(["ID"]).mean().reset_index()
group1 = stat_df["Delta"]
group2 = [0,0,0,0,0,0,]
wilcoxon(group1, group2,nan_policy= "omit")

In [ ]:
plt.figure(figsize = (0.5,1.6))
g = sns.barplot(data = delta_df.groupby(["ID"]).mean().reset_index(), y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.stripplot(data = delta_df.groupby(["ID"]).mean().reset_index(),y = "Delta",color = "black",alpha = 0.3,s = 3,jitter = 0.2)

g.set_yticks(np.arange(-0.5,0.75,0.25))
#g.set(ylim=(-0.5, 0.5))
g.set_ylabel("∆ Pupil size (z-scr)")

g.set_xlabel("V1")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/v1_delta_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###get different frequency data

id_date = { 
"004072" : ["20230721","20230718"],
"004075" : ["20230721","20230718"],
"004073" : ["20230817"],
"004109" : ["20230817","20230821"],
"004110" : ["20230817","20230821"],
"004111" : ["20230817","20230821"],
"004112" : ["20230817","20230821"]
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        laser_itis = mat_data["response"]["trial_iti"]
        laser_widths = mat_data["response"]["trial_pulse_width"]
        laser_freqs = [np.round((width/(iti + width)) * 100) for iti,width in zip(laser_itis,laser_widths)]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j,"frequency":str(int(laser_freqs[j]))}
            df_list.append(new_dict)

freq_df = pd.DataFrame.from_dict(df_list)

In [ ]:
id_date = { 
"004072" : ["20230717"],
"004075" : ["20230717","20230825"],
"004073" : ["20230808","20230811"],
"004109" : ["20230808","20230811"],
"004110" : ["20230808","20230815"],
"004111" : ["20230815"],
"004112" : ["20230811","20230815"]
}

###20hz frequency
sess =1
pre = 10
post = 10   
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]


        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])
            
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j,"frequency":"20"}
            df_list.append(new_dict)

freq1_df = pd.DataFrame.from_dict(df_list)


In [ ]:
freq_df = pd.concat([freq_df,freq1_df])

In [ ]:
plt.figure(figsize = (1.6,1.2))
g = sns.barplot(data = freq_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black",order = ["5","10","20","30"])
sns.lineplot(data = freq_df.groupby(["ID",'frequency']).mean().reset_index(), x = "frequency", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[4:]:
    y1,y2,y3,y4 = line.get_ydata()
    line.set_ydata([y4,y1,y2,y3])
g.set_yticks(np.arange(-1,0.5,0.25))
g.set(ylim=(-1, 0.25))
g.set_ylabel("∆ Pupil Size\n(z-scr)")
g.set_xlabel("Laser Freq. (Hz)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_freq_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats for freq deltas
q1 = freq_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '5'")["Delta"]
q2 = freq_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '10'")["Delta"]
q3 = freq_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '20'")["Delta"]
q4 = freq_df.groupby(["ID",'frequency']).mean().reset_index().query("frequency == '30'")["Delta"]
kruskal(q1,q2,q3,q4)

In [ ]:
### vgat different durations

id_date = { 
"004072" : ["20230720"],
"004075" : ["20230720"],
"004109" : ["20230822","20230824"],
"004110" : ["20230906"],
"004111" : ["20230824"],
"004112" : ["20230822","20230824"],
}

sess =1
pre = 10
post = 10
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        laser_durs = mat_data["response"]["trial_dur"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])

            duration_frames = int((laser_durs[j]/1000) *20)
            post_laser = np.nanmean(trial[200 + (duration_frames-10):200 + duration_frames])
            
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j,"duration":str(int(laser_durs[j])/1000)}
            df_list.append(new_dict)

dur_df = pd.DataFrame.from_dict(df_list)


In [ ]:
id_date = { 
"004072" : ["20230717"],
"004075" : ["20230717","20230825"],
"004073" : ["20230808","20230811"],
"004109" : ["20230808","20230811"],
"004110" : ["20230808","20230815"],
"004111" : ["20230815"],
"004112" : ["20230811","20230815"]
}

###20hz frequency
sess =1
pre = 10
post = 10   
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]


        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            duration_frames = int((laser_durs[j]/1000) *20)
            post_laser = np.nanmean(trial[200 + (duration_frames-10):200 + duration_frames])
            
            new_dict = {"Delta" : post_laser-pre_laser,"ID" : id,'Condition':"VGAT","Date":date,"trial":j,"duration":"5"}
            df_list.append(new_dict)

dur1_df = pd.DataFrame.from_dict(df_list)


In [ ]:
dur1_df

In [ ]:
dur_df = pd.concat([dur_df,dur1_df])

In [ ]:
dur_df = dur_df.query("ID != '004073'")

In [ ]:
dur_df

In [ ]:
plt.figure(figsize = (1.6,1.2))
g = sns.barplot(data = dur_df.groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",errorbar = "se",fill = None,errwidth= 1,edgecolor = "black",order = ["0.5","1.0","2.0","5"])
sns.lineplot(data = dur_df.groupby(["ID",'duration']).mean().reset_index(), x = "duration", y = "Delta",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_xticks(list(g.get_xticks()),labels = ["0.5","1","2","5"])

g.set_yticks(np.arange(-1,0.5,0.25))
g.set(ylim=(-1, 0.25))
g.set_ylabel("∆ Pupil Size\n(z-scr)")
g.set_xlabel("Laser Duration (s)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_dur_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats for dur deltas
q1 = dur_df.groupby(["ID",'duration']).mean().reset_index().query("duration == '0.5'")["Delta"]
q2 = dur_df.groupby(["ID",'duration']).mean().reset_index().query("duration == '1.0'")["Delta"]
q3 = dur_df.groupby(["ID",'duration']).mean().reset_index().query("duration == '2.0'")["Delta"]
q4 = dur_df.groupby(["ID",'duration']).mean().reset_index().query("duration == '5'")["Delta"]
kruskal(q1,q2,q3,q4,nan_policy="omit")

In [ ]:
###baseline pupil size vs delta

id_date = { 
"004072" : ["20230717"],
"004075" : ["20230717","20230825"],
"004073" : ["20230808","20230811"],
"004109" : ["20230808","20230811"],
"004110" : ["20230808","20230815"],
"004111" : ["20230815"],
"004112" : ["20230811","20230815"]
}
sess =1
pre = 10
post = 10
labels = list(range(1,5))
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        

        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

            new_dict = {"baseline" : binned_baseline[j],"slope" : slopes[j],"ID" : id,"Delta": post_laser - pre_laser,"Date":date,"trial":j}
            df_list.append(new_dict)
          
baseline_df = pd.DataFrame.from_dict(df_list)

In [ ]:
g = sns.pointplot(data = baseline_df, x = "baseline", y = "Delta",errorbar = "se",hue = "ID",scale = 0.4,errwidth= 1)


In [ ]:
plt.figure(figsize = (2.1,2.4))
g = sns.pointplot(data = baseline_df.groupby(["ID","baseline"]).mean().reset_index(), x = "baseline", y = "Delta",errorbar = "se",color = "dodgerblue",scale = 0.4,errwidth= 1)

# g.set_yticks(np.arange(-0.75,0.5,0.25))
# g.set_xticks(list(g.get_xticks()),labels = ["0.5","1","2","5"])

# g.set(ylim=(-0.75, 0.25))
# g.set_ylabel("∆ Pupil Size\n(z-scr)")
# g.set_xlabel("Laser Duration (s)")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_baseline_delta.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###mcherry baseline vs delta

id_date = { 
"000033" : [["20221018",2],["20221018",3]],
"000034" : [["20221014",2],["20221019",3]],
"000035" : [["20221014",2],["20221014",3]],
"000036" : [["20221014",2],["20221019",3]],

}

pre = 10
post = 10
df_list = []
labels = list(range(1,5))

for id,dates in id_date.items():
    for date,sess in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        laser_times = loadmat(matpath)["data"]["response"]["laser_time"]
        df = pd.read_csv(pupil_path)
        time = df['Time'].to_numpy()
        areas = df['Pupil Area'].to_numpy()
        trial_matrix,peri_times = trial_align(laser_times,time,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in trial_matrix:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
            
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        

        for j,trial in enumerate(trial_matrix):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

            new_dict = {"baseline" : binned_baseline[j],"slope":slopes[j],"ID" : id,"Delta": post_laser - pre_laser,"Date":date,"trial":j}
            df_list.append(new_dict)

           

mcherry_baseline_df = pd.DataFrame.from_dict(df_list)


In [ ]:
###baseline pupil size vs delta

id_date = { 
"004259": ["20240513"],
"004260": ["20240513"],
"004261": ["20240513"],
"004262": ["20240513"],
"004263": ["20240513"],
"004264": ["20240513"],
}
sess =1
pre = 10
post = 10
labels = list(range(1,5))
df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()
        areas = pupil_df['Pupil Area'].to_numpy()
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]

        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        baselines = []
        slopes = []
        for trial in pupil_snips:
            pre_laser = np.nanmean(trial[160:200])
            baselines.append(pre_laser)
            p = gaussian_filter1d(trial[190:200],1)
            p_deriv = np.gradient(p)
            slope = np.mean(p_deriv)
            slopes.append(slope)
        binned_baseline = np.array(pd.qcut(baselines,q = 4,labels = labels))
        

        for j,trial in enumerate(pupil_snips):
            pre_laser = np.nanmean(trial[160:200])
            post_laser = np.nanmean(trial[280:300])

            new_dict = {"baseline" : binned_baseline[j],"slope" : slopes[j],"ID" : id,"Delta": post_laser - pre_laser,"Date":date,"trial":j}
            df_list.append(new_dict)
          
mcherry_baseline_df1 = pd.DataFrame.from_dict(df_list)

In [ ]:
mcherry_baseline_df = pd.concat([mcherry_baseline_df,mcherry_baseline_df1])

In [ ]:
mcherry_baseline_df["ID"].unique()

In [ ]:
plt.figure(figsize = (1.2,1.2))
g = sns.pointplot(data = mcherry_baseline_df.groupby(["ID","baseline"]).mean().reset_index(), x = "baseline", y = "Delta",errorbar = "se",color = "lightgrey",scale = 0.4,errwidth=1)
g = sns.pointplot(data = baseline_df.groupby(["ID","baseline"]).mean().reset_index(), x = "baseline", y = "Delta",errorbar = "se",color = "dodgerblue",scale = 0.4,errwidth= 1)
g.axhline(0,linestyle = "dashed",linewidth = 1,color = "black",alpha  = 0.5)

g.set_yticks(np.arange(-1.5,1,0.5))

g.set(ylim=(-1.5, 0.5))
g.set_ylabel("∆ Pupil size (z-scr)")
g.set_xlabel("Pupil size\nquartile")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_baseline_delta.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###two way anova
mcherry_baseline_df["Condition"] = "mCherry"
baseline_df["Condition"] = "VGAT"
anova_df = pd.concat([mcherry_baseline_df,baseline_df])

In [ ]:
anova_df.groupby(["ID","Condition","baseline"]).mean().reset_index()

In [ ]:
pg.mixed_anova(dv='Delta', between='Condition', within='baseline', subject='ID', data=anova_df.groupby(["ID","Condition","baseline"]).mean().reset_index())


In [ ]:
anova_df.groupby(["ID","Condition","baseline"]).mean().reset_index().to_csv("S1anova.csv")

In [ ]:
model = ols('Delta ~ C(Condition) + C(baseline) + C(baseline):C(Condition)', data=anova_df).fit()
sm.stats.anova_lm(model, typ=2)

In [ ]:
###dff responses over time
def label_trial(row):
    if row['slope']<-0.02:
      return 'Constricting'
    if row['slope']>0.02:
      return 'Dilating'
    else:
      return "Stable"

baseline_df['slope_bin'] = baseline_df.apply(label_trial, axis=1)
mcherry_baseline_df['slope_bin'] = mcherry_baseline_df.apply(label_trial, axis=1)

In [ ]:
plt.figure(figsize = (1.2,1.5))
g = sns.pointplot(data = mcherry_baseline_df.groupby(["ID","slope_bin"]).mean().reset_index(), x = "slope_bin", y = "Delta",errorbar = "se",color = "lightgrey",scale = 0.4,errwidth=1)
g = sns.pointplot(data = baseline_df.groupby(["ID","slope_bin"]).mean().reset_index(), x = "slope_bin", y = "Delta",errorbar = "se",color = "dodgerblue",scale = 0.4,errwidth= 1)
g.axhline(0,linestyle = "dashed",linewidth = 1,color = "black",alpha  = 0.5)

g.set_yticks(np.arange(-1.5,1,0.5))

g.set(ylim=(-1.5, 0.5))
g.set_ylabel("∆ Pupil size (z-scr)")
#g.set_xlabel("Baseline pupil slope")

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_baseline_delta.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###closed-loop opto helper function
def live_aligned_df(mouse_id,date,sess = "1",mac = True,smooth = False,pre = 10,post = 10):
    path_start = r"C:\Users\hudal\Box\HUDA_LAB_DATA\Scripts\DLC-Live\Live_Opto\Sessions\\"
    if mac:
        path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"
    try:
        df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_areas.csv",names= ["Area","Time"])
        dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_dilations.csv",names = ["Times"])
        lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_laser.csv",names = ["Times"])["Times"].tolist()
        nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_nonlasers.csv",names = ["Times"])["Times"].tolist()
    except:
        df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas.csv",names= ["Area","Time"])
        dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations.csv",names = ["Times"])
        lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser.csv",names = ["Times"])["Times"].tolist()
        nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers.csv",names = ["Times"])["Times"].tolist()

    dil_times = dilation_df["Times"].tolist()
    Area = df["Area"].tolist()
    Area =[x if x != 0 else np.nan for x in Area]
    if smooth:
        Area = np.convolve(Area, np.ones(15)/15, mode='same')

    Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]
    Dil_times = [i - df["Time"].tolist()[0] for i in dil_times]
    Lasers = [i - df["Time"].tolist()[0] for i in lasers]
    Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
    nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
    nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
    Time = np.arange(0,Time[-1],0.05)
    #Time = [np.round(x,2) for x in Time]
    area = np.interp(Time, nan_removed_times, nan_removed_area)
    Area = zscore(area)
    #plt.plot(Time,Area)
    Lasers = np.array(Lasers)
    print(len(Lasers))
    Nonlasers = np.array(Nonlasers)
    print(len(Nonlasers))
    Time = np.array(Time)
    Area = np.array(Area)
    laser_matrix,dil_times = trial_align(Lasers,Time,Area,fps = 20,pre = pre, post = post)
    df_list = []
    for j,trial in enumerate(laser_matrix):
        ##check if slope during trial is positive
        p = gaussian_filter1d(trial,20)
        p_deriv = np.gradient(p,dil_times)
        slope = np.mean(p_deriv[160:200])
        if slope >0:
            # baseline = np.nanmean(trial[0:100])
            # trial = trial - baseline
            for i,time in enumerate(dil_times):
                new_dict = {"Time from Stim (s)" : np.round(time,2), "trial" : j, "Pupil Area" : trial[i],"Type":"Laser Trial","ID":mouse_id,"Date":date}
                df_list.append(new_dict)

    nonlaser_matrix,dil_times = trial_align(Nonlasers,Time,Area,fps = 20,pre = pre, post = post)
    for j,trial in enumerate(nonlaser_matrix):
        ##check if slope during trial is positive
        p = gaussian_filter1d(trial,20)
        p_deriv = np.gradient(p,dil_times)
        slope = np.mean(p_deriv[160:200])
        if slope >0:
            # baseline = np.nanmean(trial[0:100])
            # trial = trial - baseline
            for i,time in enumerate(dil_times):
                new_dict = {"Time from Stim (s)" : np.round(time,2), "trial" : j, "Pupil Area" : trial[i],"Type":"Non-Laser Trial","ID":mouse_id,"Date":date}
                df_list.append(new_dict)
    aligned_df = pd.DataFrame.from_dict(df_list)
    return aligned_df


In [ ]:
###25% threshold

id_date = { 
"004072" : ["20230727","20230726"],
"004075" : ["20230727","20230726"],
"004109" : ["20230907","20230908"],
"004112" : ["20230908"],
"004110" : ["20230911"],
"004111" : ["20230911"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = live_aligned_df(id,date,"1",mac = True,post = 15)
        df_list.append(df)
df = pd.concat(df_list)
live_df = df.reset_index()

In [ ]:
ax = sns.relplot(data = live_df, x = "Time from Stim (s)", y = "Pupil Area",errorbar = "se",palette = ["dodgerblue","black"],linewidth = 1,hue = "Type",legend= None,col = "ID",kind="line",row = "Date")


In [ ]:
plt.figure(figsize = (1.9,1.6))

ax = sns.lineplot(data = live_df.groupby(["ID","Type","Time from Stim (s)"]).mean().reset_index(), x = "Time from Stim (s)", y = "Pupil Area",errorbar = "se",palette = ["dodgerblue","black"],linewidth = 1,hue = "Type",legend= None)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.2,1.6],2,0.2)
ax.set_xlabel("Time from laser (s)")
ax.set_ylabel("Pupil size (z-scr)")
#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/live_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###v1 control

id_date = { 
"004253" : ["20240520"],
"004254" : ["20240521"],
"004255" : ["20240521"],
"004256" : ["20240521"],
"004257" : ["20240521"],
"004258" : ["20240521"],
 }


df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = live_aligned_df(id,date,"1",mac = True,post = 15)
        df_list.append(df)
df = pd.concat(df_list)
v1_live_df = df.reset_index()

In [ ]:
###mcherry control

id_date = { 
"004259" : ["20240514"],
"004260" : ["20240514","20240517"],
"004261" : ["20240514"],
"004262" : ["20240514","20240522"],
"004263" : ["20240522"],
"004264" : ["20240514"],
 }



df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = live_aligned_df(id,date,"1",mac = True,post = 15)
        df_list.append(df)
df = pd.concat(df_list)
mcherry_live_df = df.reset_index()

In [ ]:
plt.figure(figsize = (1.6,1.6))

ax = sns.lineplot(data = mcherry_live_df.groupby(["ID","Type","Time from Stim (s)"]).mean().reset_index(), x = "Time from Stim (s)", y = "Pupil Area",errorbar = "se",palette = ["dodgerblue","black"],linewidth = 1,hue = "Type",legend= None)
ax.axvspan(0,5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.5,2],2,0.5)
ax.set_xlabel("Time from detection (s)")
ax.set_ylabel("Pupil size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_live_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
###closed loop bar graph function
bar_df_list = []
for ID in live_df.ID.unique():
    mouse_df = live_df.query(f"ID =='{ID}'")
    for date in mouse_df.Date.unique():
        date_df = mouse_df.query(f"Date =='{date}'")
        for type in date_df.Type.unique():
            type_df = date_df.query(f"Type =='{type}'")
            for trial in type_df.trial.unique():
                trial_df = type_df.query(f"trial =={trial}")
                rslt_df = trial_df.loc[trial_df['Time from Stim (s)'] > 0]
                rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
                time = rslt_df["Time from Stim (s)"]
                area = np.array(rslt_df["Pupil Area"])
                detection_area = area[0]
                peak = max(area[:60])
                peak_ix = np.argmax(area[:60])
                p_20 = peak * 0.2
                # if type == "Non-Laser Trial":
                #     plt.plot(time,area,color = "black")
                #     plt.scatter(peak_ix/20,peak)
                # else:
                #     plt.plot(time,area)
                #     plt.scatter(peak_ix/20,peak)

                # plt.show()
                # plt.clf()

                p_20_ix = np.absolute(area[peak_ix:]-p_20).argmin() + peak_ix

                time_to_20 = (p_20_ix-peak_ix)/20


                new_dict = {"AUC" : np.trapz(area,time),"ID" : ID,"type": type,"Date":date,"Detection Area":detection_area,"T20":time_to_20,"Peak":peak,"Time to Peak":(peak_ix/20)}
                bar_df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###closed loop auc bar
#plt.figure(figsize = (0.7,0.7))
g = sns.barplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Time to Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Time to Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 5))
g.xaxis.label.set_visible(False)



In [ ]:
###closed loop auc bar
plt.figure(figsize = (1.2,1.6))
g = sns.barplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_auc_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
##closed loop t20 bar
plt.figure(figsize = (1.2,1.6))
g = sns.barplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "T20",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "T20",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,6,1))
g.set(ylim=(0, 5))
g.set_ylabel("Time to 50%\n of Peak")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_t20_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = bar_df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["T20"], group2["T20"],nan_policy= "omit")

In [ ]:
###Area at detection
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 2))
g.set_ylabel("Pupil Size\nat Detection")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_detectionarea_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###Area peak
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Peak Pupil Size\n (z-scr)")
g.xaxis.label.set_visible(False)

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_peakarea_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = bar_df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["AUC"], group2["AUC"],nan_policy= "omit")

In [ ]:
###time to peak for on animal averaged traces
bar_df_list = []
for ID in live_df.ID.unique():
    mouse_df = live_df.query(f"ID =='{ID}'")
    for type in mouse_df.Type.unique():
        type_df = mouse_df.query(f"Type =='{type}'")
        rslt_df = type_df.loc[type_df['Time from Stim (s)'] > 0]
        rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
        print(type)
        area = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Pupil Area"])
        #area = area - np.min(area)
        time = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Time from Stim (s)"])
        peak = max(area)
        peak_ix = np.argmax(area)
        p_50 = peak*0.5
        p_50_ix = np.absolute(area[peak_ix:]-p_50).argmin() + peak_ix
        detection = area[0]

        if ID == "004072":
            plt.plot(time,area)
            plt.scatter(peak_ix/20,peak)
            plt.scatter(p_50_ix/20,area[p_50_ix])

            plt.show()
            plt.clf()
            

        time_to_50 = (p_50_ix-peak_ix)/20


        new_dict = {"ID" : ID,"type": type,"T50":time_to_50,"Peak":peak,"AUC":np.trapz(area,time),"Detection":detection}
        bar_df_list.append(new_dict)


t50_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###T50
plt.figure(figsize = (0.8,1.6))
g = sns.barplot(data = t50_df, x = "type", y = "T50",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_df, x = "type", y = "T50",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Time to 50% of peak")
g.xaxis.label.set_visible(False)
labels = ["Non\nlaser","Laser"]
g.set_xticklabels(labels)

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_t50_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
sns.lineplot(data = t50_df, x = "type", y = "T50",hue = "ID",estimator = None,linewidth = 5)


In [ ]:
###T50
plt.figure(figsize = (0.8,1.6))
g = sns.barplot(data = t50_df, x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_df, x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)
labels = ["Non\nlaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_auc_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###T50
plt.figure(figsize = (0.8,1.6))
g = sns.barplot(data = t50_df, x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_df, x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Peak pupil size (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nlaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_peak_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###pupil size at detection
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_df, x = "type", y = "Detection",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_df, x = "type", y = "Detection",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,3,1))
g.set(ylim=(0, 2))
g.set_ylabel("Pupil size\nat detection (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nlaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_detectionarea_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["T50"], group2["T50"],nan_policy= "omit")

In [ ]:
stat_df = t50_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["AUC"], group2["AUC"],nan_policy= "omit")

In [ ]:
stat_df = t50_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Peak"], group2["Peak"],nan_policy= "omit")

In [ ]:
stat_df = t50_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Detection"], group2["Detection"],nan_policy= "omit")

In [ ]:
###time to peak for on animal averaged traces V1 mice
bar_df_list = []
for ID in v1_live_df.ID.unique():
    mouse_df = v1_live_df.query(f"ID =='{ID}'")
    for type in mouse_df.Type.unique():
        type_df = mouse_df.query(f"Type =='{type}'")
        rslt_df = type_df.loc[type_df['Time from Stim (s)'] > 0]
        rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
        print(type)
        area = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Pupil Area"])
        time = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Time from Stim (s)"])
        peak = max(area)
        peak_ix = np.argmax(area)
        p_50 = peak*0.5
        p_50_ix = np.absolute(area[peak_ix:]-p_50).argmin() + peak_ix
        detection = area[0]

        # plt.plot(area)
        # plt.scatter(peak_ix,peak)
        # plt.scatter(p_50_ix,area[p_50_ix])

        # plt.show()
        # plt.clf()
        

        time_to_50 = (p_50_ix-peak_ix)/20


        new_dict = {"ID" : ID,"type": type,"T50":time_to_50,"Peak":peak,"AUC":np.trapz(area,time),"Detection":detection}
        bar_df_list.append(new_dict)


t50_v1_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_v1_df, x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_v1_df, x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,14,2))
g.set(ylim=(0, 12))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/v1_auc_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_v1_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["AUC"], group2["AUC"],nan_policy= "omit")

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_v1_df, x = "type", y = "T50",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_v1_df, x = "type", y = "T50",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Time to 50%\nof Peak")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/v1_t50_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_v1_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["T50"], group2["T50"],nan_policy= "omit")

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_v1_df, x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_v1_df, x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Peak Pupil Size\n (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/v1_peak_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_v1_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Peak"], group2["Peak"],nan_policy= "omit")

In [ ]:
###time to peak for on animal averaged traces mcherry mice
bar_df_list = []
for ID in mcherry_live_df.ID.unique():
    mouse_df = mcherry_live_df.query(f"ID =='{ID}'")
    for type in mouse_df.Type.unique():
        type_df = mouse_df.query(f"Type =='{type}'")
        rslt_df = type_df.loc[type_df['Time from Stim (s)'] > 0]
        rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
        print(type)
        area = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Pupil Area"])
        time = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Time from Stim (s)"])
        peak = max(area)
        peak_ix = np.argmax(area)
        p_50 = peak*0.5
        p_50_ix = np.absolute(area[peak_ix:]-p_50).argmin() + peak_ix
        detection = area[0]

        # plt.plot(area)
        # plt.scatter(peak_ix,peak)
        # plt.scatter(p_50_ix,area[p_50_ix])

        # plt.show()
        # plt.clf()
        

        time_to_50 = (p_50_ix-peak_ix)/20


        new_dict = {"ID" : ID,"type": type,"T50":time_to_50,"Peak":peak,"AUC":np.trapz(area,time),"Detection":detection}
        bar_df_list.append(new_dict)


t50_mcherry_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_mcherry_df, x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_mcherry_df, x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,14,2))
g.set(ylim=(0, 12))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_auc_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_mcherry_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["AUC"], group2["AUC"],nan_policy= "omit")

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_mcherry_df, x = "type", y = "T50",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_mcherry_df, x = "type", y = "T50",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Time to 50%\nof Peak")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_t50_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_mcherry_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["T50"], group2["T50"],nan_policy= "omit")

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_mcherry_df, x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_mcherry_df, x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Peak Pupil Size\n (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/mcherry_peak_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_mcherry_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Peak"], group2["Peak"],nan_policy= "omit")

In [ ]:
###closed loop different durations helper
###alignment function

###DLC-Live different durations

def dur_aligned_df(mouse_id,date,sess,mac = True):
    path_start = r"C:\Users\hudal\Box\HUDA_LAB_DATA\Scripts\DLC-Live\Live_Opto\Sessions\\"
    if mac:
        path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"
    df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas_duration.csv",names= ["Area","Time"])
    dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations_duration.csv",names = ["Times"])
    lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser_duration.csv",names = ["Times"])["Times"].tolist()
    nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers_duration.csv",names = ["Times"])["Times"].tolist()
    trial_durs = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_trials_duration.csv",names = ["Duration"])["Duration"].tolist()
    print(pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_trials_duration.csv",names = ["Duration"])["Duration"].value_counts())

    dil_times = dilation_df["Times"].tolist()
    Area = df["Area"].tolist()
    Area =[x if x != 0 else np.nan for x in Area]
    Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]
    Dil_times = [i - df["Time"].tolist()[0] for i in dil_times]
    Lasers = [i - df["Time"].tolist()[0] for i in lasers]
    Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
    nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
    nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
    Time = np.arange(0,Time[-1],0.05)
    #Time = [np.round(x,2) for x in Time]
    area = np.interp(Time, nan_removed_times, nan_removed_area)
    Area = zscore(area)
    #plt.plot(Time,Area)
    Lasers = np.array(Lasers)
    print(len(Lasers))
    Nonlasers = np.array(Nonlasers)
    print(len(Nonlasers))
    Time = np.array(Time)
    Area = np.array(Area)


    laser_matrix,dil_times = trial_align(Lasers,Time,Area,fps = 20,pre = 10, post = 10)
    df_list = []
    for j,trial in enumerate(laser_matrix):
        ##check if slope during trial is positive
        p = gaussian_filter1d(trial,20)
        p_deriv = np.gradient(p,dil_times)
        slope = np.mean(p_deriv[160:200])
        if slope >0:
            for i,time in enumerate(dil_times):
                new_dict = {"Time from Stim (s)" : np.round(time,2), "trial" : j, "Pupil Area" : trial[i],"Type":"Laser Trial","ID":mouse_id,"Date":date,"duration":trial_durs[j]}
                df_list.append(new_dict)

    nonlaser_matrix,dil_times = trial_align(Nonlasers,Time,Area,fps = 20,pre = 10, post = 10)
    for j,trial in enumerate(nonlaser_matrix):
        ##check if slope during trial is positive
        p = gaussian_filter1d(trial,20)
        p_deriv = np.gradient(p,dil_times)
        slope = np.mean(p_deriv[160:200])
        if slope >0:
            for i,time in enumerate(dil_times):
                new_dict = {"Time from Stim (s)" : np.round(time,2), "trial" : j, "Pupil Area" : trial[i],"Type":"Non-Laser Trial","ID":mouse_id,"Date":date}
                df_list.append(new_dict)
                
    aligned_df = pd.DataFrame.from_dict(df_list)
    return aligned_df

In [ ]:
###shorter duration
id_date = { 
"004075": ["20230731"],
"004072": ["20230731"],
"004109": ["20230921"],
"004112": ["20231004"],
"004111": ["20231004"],
"004110": ["20231005"]
}


df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = dur_aligned_df(id,date,"1",mac = True)
        df_list.append(df)

df = pd.concat(df_list)
live_dur_df = df.reset_index()


In [ ]:
live_short_df = pd.concat([live_dur_df.query("Type == 'Non-Laser Trial'"),live_dur_df.query("duration == 3000")]).drop(labels="duration",axis = 1)
ax = sns.relplot(data = live_short_df, x = "Time from Stim (s)", y = "Pupil Area",errorbar = "se",palette = ["black","dodgerblue"],linewidth = 1,hue = "Type",legend= True,col = "ID",kind="line",row = "Date")


In [ ]:


plt.figure(figsize = (1.6,1.6))
ax = sns.lineplot(data = live_short_df.groupby(["ID","Type","Time from Stim (s)"]).mean().reset_index(), x = "Time from Stim (s)", y = "Pupil Area",errorbar = "se",palette = ["dodgerblue","black"],linewidth = 1,hue = "Type",legend= None)
ax.axvspan(0,0.5,alpha = 0.2)
format_ax(ax,[-2,10],[-0.2,1.6],2,0.2)
ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/live_short_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###closed loop bar graph function
bar_df_list = []
for ID in live_short_df.ID.unique():
    mouse_df = live_short_df.query(f"ID =='{ID}'")
    for date in mouse_df.Date.unique():
        date_df = mouse_df.query(f"Date =='{date}'")
        for type in date_df.Type.unique():
            type_df = date_df.query(f"Type =='{type}'")
            for trial in type_df.trial.unique():
                trial_df = type_df.query(f"trial =={trial}")
                rslt_df = trial_df.loc[trial_df['Time from Stim (s)'] > 0]
                rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
                time = rslt_df["Time from Stim (s)"]
                area = np.array(rslt_df["Pupil Area"])
                detection_area = area[0]
                peak = max(area)
                peak_ix = np.argmax(area)
                p_20 = peak * 0.2

                p_20_ix = np.absolute(area[peak_ix:]-p_20).argmin() + peak_ix

                time_to_20 = (p_20_ix-peak_ix)/20


                new_dict = {"AUC" : np.trapz(area,time),"ID" : ID,"type": type,"Date":date,"Detection Area":detection_area,"T20":time_to_20,"Peak":peak,"Time to Peak":(peak_ix/20)}
                bar_df_list.append(new_dict)


short_bar_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###closed loop auc bar
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,15,3))
g.set(ylim=(0, 12))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_auc_shortbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
##closed loop t20 bar
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "T20",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "T20",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,6,1))
g.set(ylim=(0, 5))
g.set_ylabel("Time to 20%\n of Peak")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_t20_shortbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###Area at detection
plt.figure(figsize = (1.2,0.9))
g = sns.barplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 2))
g.set_ylabel("Pupil Size\nat Detection")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_detectionarea_shortbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###Area peak
plt.figure(figsize = (0.9,0.9))
g = sns.barplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = short_bar_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,5,1))
g.set(ylim=(0, 4))
g.set_ylabel("Peak Pupil Size\n (z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_peakarea_shortbar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = short_bar_df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Peak"], group2["Peak"],nan_policy= "omit")

In [ ]:
live_short_df["Type"].value_counts()/400

In [ ]:
###area at detection for all experiments

###Area at detection
plt.figure(figsize = (1.2,0.9))
test = pd.concat([short_bar_df,bar_df])
g = sns.barplot(data = test.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = test.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Detection Area",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])
g.set_yticks(np.arange(0,3,1))
g.set(ylim=(0, 2))
g.set_ylabel("Pupil Size\nat Detection\n(z-scr)")
g.xaxis.label.set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_detectionarea_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
test.groupby(["ID",'type']).mean().reset_index()

In [ ]:
###time to peak for on animal averaged traces
bar_df_list = []
for ID in live_short_df.ID.unique():
    mouse_df = live_short_df.query(f"ID =='{ID}'")
    for type in mouse_df.Type.unique():
        type_df = mouse_df.query(f"Type =='{type}'")
        rslt_df = type_df.loc[type_df['Time from Stim (s)'] > 0]
        rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
        print(type)
        area = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Pupil Area"])
        time = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Time from Stim (s)"])
        peak = max(area)
        peak_ix = np.argmax(area)
        p_50 = peak*0.5
        p_50_ix = np.absolute(area[peak_ix:]-p_50).argmin() + peak_ix

        # plt.plot(area)
        # plt.scatter(peak_ix,peak)
        # plt.scatter(p_50_ix,area[p_50_ix])

        # plt.show()
        # plt.clf()
        

        time_to_50 = (p_50_ix-peak_ix)/20


        new_dict = {"ID" : ID,"type": type,"T50":time_to_50,"Peak":peak,"AUC":np.trapz(area,time)}
        bar_df_list.append(new_dict)


t50_short_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_short_df, x = "type", y = "T50",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_short_df, x = "type", y = "T50",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)


g.set_yticks(np.arange(0,12,2))
g.set(ylim=(0, 10))
g.set_ylabel("Time to 50%\nof Peak")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_t50short_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_short_df, x = "type", y = "Peak",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_short_df, x = "type", y = "Peak",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)


g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Peak Pupil Size\n (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_peakshort_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###T50
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = t50_short_df, x = "type", y = "AUC",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = t50_short_df, x = "type", y = "AUC",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)


g.set_yticks(np.arange(0,14,2))
g.set(ylim=(0, 12))
g.set_ylabel("Pupil AUC")
g.xaxis.label.set_visible(False)
labels = ["Non\nLaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_aucshort_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = t50_short_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Peak"], group2["Peak"],nan_policy= "omit")

In [ ]:
stat_df = t50_short_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["AUC"], group2["AUC"],nan_policy= "omit")

In [ ]:
stat_df = t50_short_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["T50"], group2["T50"],nan_policy= "omit")

In [ ]:
###example calibration

mouse_id = "004072"
date = "20230726"
df = pd.read_csv(fr"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/{mouse_id}_{date}_calibrate.csv",names= ["Area","Time"])
Area = df["Area"].tolist()
Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]


nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
Area = np.interp(Time, nan_removed_times, nan_removed_area)
raw_Area = Area
Area = zscore(Area)
prom = 2
peak_times = [Time[i] for i in list(find_peaks(Area,prominence= prom)[0])]
peak_y = [Area[i] for i in list(find_peaks(Area,prominence= prom)[0])]

plt.figure(figsize = (5.3,0.8))
plt.plot(Time,Area,linewidth = 1,color = "dodgerblue")
plt.scatter(peak_times,peak_y,s = 60, color="none", edgecolor="black",linewidth = 0.5)

heights = find_peaks(Area,prominence= prom)[1]["prominences"]
z = np.mean(heights) * 0.25
plt.axhline(z,linewidth = 1,color = "black",linestyle = "dashed")
plt.xlim(115,215)
plt.ylim(-2,5)


ax1 = plt.gca()
ax1.plot([115,120], [-1, -1],color = "black",linewidth = 1)
ax1.plot([120,120], [0, 2],color = "dodgerblue",linewidth = 1)

ax1.spines[['bottom']].set_visible(False)
ax1.spines[['left']].set_visible(False)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/example_calibration.pdf",format ="pdf",transparent = True)


In [ ]:
###example session

mouse_id = "004072"
date = "20230726"
sess = 1
path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"

df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas.csv",names= ["Area","Time"])
dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations.csv",names = ["Times"])
lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser.csv",names = ["Times"])["Times"].tolist()
nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers.csv",names = ["Times"])["Times"].tolist()


dil_times = dilation_df["Times"].tolist()
Area = df["Area"].tolist()
Area =[x if x != 0 else np.nan for x in Area]
Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]
Dil_times = [i - df["Time"].tolist()[0] for i in dil_times]
Lasers = [i - df["Time"].tolist()[0] for i in lasers]
Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
Time = np.arange(0,Time[-1],0.05)
area = np.interp(Time, nan_removed_times, nan_removed_area)
Area = zscore(area)
Lasers = np.array(Lasers)
Nonlasers = np.array(Nonlasers)
Time = np.array(Time)
Area = np.array(Area)
prom = 2
# peak_times = [Time[i] for i in list(find_peaks(Area,prominence= prom)[0])]
# peak_y = [Area[i] for i in list(find_peaks(Area,prominence= prom)[0])]
# plt.plot(Time,Area)
# plt.scatter(peak_times,peak_y,s = 100, color="none", edgecolor="blue")
# heights = find_peaks(Area,prominence= prom)[1]["prominences"]
# # z = np.mean(heights) * 0.25
# # plt.axhline(z,linestyle = "dashed")
# # plt.xlim(110,220)
# # plt.ylim(-2,5)

In [ ]:

plt.figure(figsize = (5.3,0.8))
plt.plot(Time,Area,color = "dodgerblue",linewidth = 1)
for time in Lasers:
    plt.gca().axvline(time,color = "blue",linewidth = 1,alpha = 0.6)

for time in Nonlasers:
    plt.gca().axvline(time,color = "black",linewidth = 1,alpha = 0.6)

plt.xlim(1110,1210)
plt.ylim(-2,5)
plt.axhline(z,linewidth = 1,color = "black",linestyle = "dashed")

ax1 = plt.gca()
ax1.spines[['bottom']].set_visible(False)
ax1.spines[['left']].set_visible(False)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/example_session.pdf",format ="pdf",transparent = True)

In [ ]:
###expected vs actual trials

id_date = { 
"004072" : ["20230727","20230726"],
"004075" : ["20230727","20230726"],
"004109" : ["20230907","20230908"],
"004112" : ["20230908"],
"004110" : ["20230911"],
"004111" : ["20230911"]
}


In [ ]:
###example calibration

mouse_id = "004264"
date = "20240514"
df = pd.read_csv(fr"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/{mouse_id}_{date}_calibrate.csv",names= ["Area","Time"])
Area = df["Area"].tolist()
Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]


nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
Area = np.interp(Time, nan_removed_times, nan_removed_area)
raw_Area = Area
Area = zscore(Area)
prom = 2
peak_times = [Time[i] for i in list(find_peaks(Area,prominence= prom)[0])]
peak_y = [Area[i] for i in list(find_peaks(Area,prominence= prom)[0])]

#plt.plot(Time,Area,linewidth = 1,color = "dodgerblue")
#plt.scatter(peak_times,peak_y,s = 60, color="none", edgecolor="black",linewidth = 0.5)

heights = find_peaks(Area,prominence= prom)[1]["prominences"]
z = np.mean(heights) * 0.25
#plt.axhline(z,linewidth = 1,color = "black",linestyle = "dashed")

std = np.std(raw_Area)
mean = np.mean(raw_Area)
pixel_thresh = z * std + mean








In [ ]:
print(pixel_thresh)
print(z)


In [ ]:
plt.plot(Time,raw_Area,linewidth = 1,color = "dodgerblue")
plt.axhline(pixel_thresh,linewidth = 1,color = "black",linestyle = "dashed")


In [ ]:
###example session

mouse_id = "004072"
date = "20230727"
sess = 1
path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"

df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas.csv",names= ["Area","Time"])
dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations.csv",names = ["Times"])
lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser.csv",names = ["Times"])["Times"].tolist()
nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers.csv",names = ["Times"])["Times"].tolist()


dil_times = dilation_df["Times"].tolist()
Area = df["Area"].tolist()
Area =[x if x != 0 else np.nan for x in Area]
Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]
Dil_times = [i - df["Time"].tolist()[0] for i in dil_times]
Lasers = [i - df["Time"].tolist()[0] for i in lasers]
Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
Time = np.arange(0,Time[-1],0.05)
area = np.interp(Time, nan_removed_times, nan_removed_area)
Area = zscore(area)
raw_area = area
Lasers = np.array(Lasers)
Nonlasers = np.array(Nonlasers)
Time = np.array(Time)
Area = np.array(Area)
prom = pixel_thresh

plt.plot(Time,raw_area)
peaks = find_peaks(raw_area,prominence= pixel_thresh)


In [ ]:
###cl
def area_at_detection(mouse_id,date,sess = "1",mac = True,smooth = False,pre = 10,post = 10):
    path_start = r"C:\Users\hudal\Box\HUDA_LAB_DATA\Scripts\DLC-Live\Live_Opto\Sessions\\"
    if mac:
        path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"
    try:
        df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_areas.csv",names= ["Area","Time"])
        dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_dilations.csv",names = ["Times"])
        lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_laser.csv",names = ["Times"])["Times"].tolist()
        nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_nonlasers.csv",names = ["Times"])["Times"].tolist()
    except:
        df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas.csv",names= ["Area","Time"])
        dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations.csv",names = ["Times"])
        lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser.csv",names = ["Times"])["Times"].tolist()
        nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers.csv",names = ["Times"])["Times"].tolist()

    dil_times = dilation_df["Times"].tolist()
    Area = df["Area"].tolist()
    Area =[x if x != 0 else np.nan for x in Area]
    if smooth:
        Area = np.convolve(Area, np.ones(15)/15, mode='same')

    Time = [i - df["Time"].tolist()[0] for i in df["Time"].tolist()]
    Dil_times = [i - df["Time"].tolist()[0] for i in dil_times]
    Lasers = [i - df["Time"].tolist()[0] for i in lasers]
    Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
    nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
    nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
    Time = np.arange(0,Time[-1],0.05)
    #Time = [np.round(x,2) for x in Time]
    area = np.interp(Time, nan_removed_times, nan_removed_area)
    Area = zscore(area)
    #plt.plot(Time,Area)
    Time = np.array(Time)
    Area = np.array(Area)
    laser_matrix,dil_times = trial_align(Dil_times,Time,Area,fps = 20,pre = pre, post = post)
    pixel_matrix,pixel_times = trial_align(Dil_times,Time,area,fps = 20,pre = pre, post = post)


    df_list = []
    for j,trial in enumerate(laser_matrix):
        pixel_trial = pixel_matrix[j]
        pixel_at_detection = pixel_trial[200]
     
        area_at_detection = trial[200]
        new_dict = { "Detection Area" : area_at_detection,"ID":mouse_id,"Date":date,"Pixel Area" : pixel_at_detection}
        df_list.append(new_dict)


    aligned_df = pd.DataFrame.from_dict(df_list)
    return aligned_df


In [ ]:
###threshold df

# id_date = { 
# "004072" : ["20230727","20230726"],
# "004075" : ["20230727","20230726"],
# "004109" : ["20230907","20230908"],
# "004112" : ["20230908"],
# "004110" : ["20230911"],
# "004111" : ["20230911"]
# }

# id_date = { 
# "004253" : ["20240520"],
# "004254" : ["20240521"],
# "004255" : ["20240521"],
# "004256" : ["20240521"],
# "004257" : ["20240521"],
# "004258" : ["20240521"],
#  }

id_date = { 
"004259" : ["20240514"],
"004260" : ["20240514"],
"004261" : ["20240514"],
"004262" : ["20240514"],
"004263" : ["20240522"],
"004264" : ["20240514"],
 }


df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = area_at_detection(id,date,"1",mac = True,post = 15)
        df_list.append(df)
df = pd.concat(df_list)
thresh_df = df.reset_index()

In [ ]:
thresh_df["Pixel Area"].mean()

In [ ]:
thresh_df.groupby(["ID","Date"]).mean().reset_index()

In [ ]:
###get threshold validation data

thresh_data = pd.read_csv("thresh_df.csv")
thresh_data

In [ ]:
pivot_thresh = pd.pivot_table(thresh_data,values="Pupil Size (pixels)",index = "Mouse ID", columns = ["Type"]).reset_index()

In [ ]:
pivot_thresh

In [ ]:
###calculate percent change
def percentage_change(col1,col2):
    return ((col2 - col1) / col1) * 100

pivot_thresh["percent change"] = percentage_change(pivot_thresh["Expected"],pivot_thresh["Actual"])

In [ ]:
pivot_thresh["percent change"].mean()

In [ ]:
pivot_thresh["percent change"].sem()

In [ ]:
pivot_thresh

In [ ]:
###pupil size at detection validation
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = thresh_data.groupby(["Mouse ID","Type"]).mean().reset_index(), x = "Type", y = "Pupil Size (pixels)",errorbar = "se",fill = None,errwidth= 1,order = ["Expected","Actual"])
sns.lineplot(data = thresh_data.groupby(["Mouse ID","Type"]).mean().reset_index(), x = "Type", y = "Pupil Size (pixels)",units = "Mouse ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,5000,1000))
g.set(ylim=(0, 3000))
g.set_ylabel("Pupil size (pixels)")
g.xaxis.label.set_visible(False)
#labels = ["Non\nLaser","Laser"]
#g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_expectedactual_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = thresh_data.groupby(["Mouse ID","Type"]).mean().reset_index()
group1 = stat_df[stat_df['Type']=="Expected"]
group2 = stat_df[stat_df['Type']=="Actual"]
wilcoxon(group1["Pupil Size (pixels)"], group2["Pupil Size (pixels)"],nan_policy= "omit")

In [ ]:
group1["Mouse ID"].value_counts().sum()

In [ ]:
###all mice

id_date = { 
"004072" : ["20230727","20230726"],
"004075" : ["20230727","20230726"],
"004109" : ["20230907","20230908"],
"004112" : ["20230908"],
"004110" : ["20230911"],
"004111" : ["20230911"],

"004253" : ["20240520"],
"004254" : ["20240521"],
"004255" : ["20240521"],
"004256" : ["20240521"],
"004257" : ["20240521"],
"004258" : ["20240521"],

"004259" : ["20240514"],
"004260" : ["20240514","20240517"],
"004261" : ["20240514"],
"004262" : ["20240514","20240522"],
"004263" : ["20240522"],
"004264" : ["20240514"],
 }


df_list = []
for id,dates in id_date.items():
    for date in dates:
        print(id,date)
        df = live_aligned_df(id,date,"1",mac = True,post = 15)
        df_list.append(df)
df = pd.concat(df_list)
all_live_df = df.reset_index()

In [ ]:
###area at detection for laser vs non laser for all mice
bar_df_list = []
for ID in all_live_df.ID.unique():
    mouse_df = all_live_df.query(f"ID =='{ID}'")
    for type in mouse_df.Type.unique():
        type_df = mouse_df.query(f"Type =='{type}'")
        rslt_df = type_df.loc[type_df['Time from Stim (s)'] > 0]
        rslt_df = rslt_df.loc[rslt_df['Time from Stim (s)'] < 10]
        print(type)
        area = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Pupil Area"])
        time = np.array(rslt_df.groupby(["Time from Stim (s)"]).mean().reset_index()["Time from Stim (s)"])
        peak = max(area)
        peak_ix = np.argmax(area)
        p_50 = peak*0.5
        p_50_ix = np.absolute(area[peak_ix:]-p_50).argmin() + peak_ix
        detection = area[0]

        # plt.plot(area)
        # plt.scatter(peak_ix,peak)
        # plt.scatter(p_50_ix,area[p_50_ix])

        # plt.show()
        # plt.clf()
        

        time_to_50 = (p_50_ix-peak_ix)/20


        new_dict = {"ID" : ID,"type": type,"T50":time_to_50,"Peak":peak,"AUC":np.trapz(area,time),"Detection":detection}
        bar_df_list.append(new_dict)

all_df = pd.DataFrame.from_dict(bar_df_list)

In [ ]:
all_df

In [ ]:
###pupil size at detection
plt.figure(figsize = (1.6,1.6))
g = sns.barplot(data = all_df, x = "type", y = "Detection",errorbar = "se",fill = None,errwidth= 1,order = ["Non-Laser Trial", "Laser Trial"])
sns.lineplot(data = all_df, x = "type", y = "Detection",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)
for line in g.get_lines()[2:]:
    y1,y2 = line.get_ydata()
    line.set_ydata([y2,y1])

g.set_yticks(np.arange(0,3,1))
g.set(ylim=(0, 2))
g.set_ylabel("Pupil size\nat detection (z-scr)")
g.xaxis.label.set_visible(False)
labels = ["Non\nlaser","Laser"]
g.set_xticklabels(labels)

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/vgat_detectionarea_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
stat_df = all_df
group1 = stat_df[stat_df['type']=="Laser Trial"]
group2 = stat_df[stat_df['type']=="Non-Laser Trial"]
wilcoxon(group1["Detection"], group2["Detection"],nan_policy= "omit")

In [ ]:
###closed loop delay
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


id_date = { 
"004072" : ["20230727","20230726"],
"004075" : ["20230727","20230726"],
"004109" : ["20230907","20230908"],
"004112" : ["20230908"],
"004110" : ["20230911"],
"004111" : ["20230911"],

"004253" : ["20240520"],
"004254" : ["20240521"],
"004255" : ["20240521"],
"004256" : ["20240521"],
"004257" : ["20240521"],
"004258" : ["20240521"],

"004259" : ["20240514"],
"004260" : ["20240514","20240517"],
"004261" : ["20240514"],
"004262" : ["20240514","20240522"],
"004263" : ["20240522"],
"004264" : ["20240514"],


 }
df_list = []



for mouse_id,dates in id_date.items():
    for date in dates:
        path_start = "/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Scripts/DLC-Live/Live_Opto/Sessions/"
        try:
            df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_areas.csv",names= ["Area","Time"])
            dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_dilations.csv",names = ["Times"])
            lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_laser.csv",names = ["Times"])["Times"].tolist()
            nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_nonlasers.csv",names = ["Times"])["Times"].tolist()
        except:
            df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_areas.csv",names= ["Area","Time"])
            dilation_df = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_dilations.csv",names = ["Times"])
            lasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_laser.csv",names = ["Times"])["Times"].tolist()
            nonlasers = pd.read_csv(fr"{path_start}{mouse_id}_{date}_{sess}_nonlasers.csv",names = ["Times"])["Times"].tolist()

        dil_times = dilation_df["Times"].tolist()
        Area = df["Area"].tolist()
        Area =[x if x != 0 else np.nan for x in Area]

        Time = np.array([i - df["Time"].tolist()[0] for i in df["Time"].tolist()])
        Dil_times = np.array([i - df["Time"].tolist()[0] for i in dil_times])
        Lasers = [i - df["Time"].tolist()[0] for i in lasers]
        Nonlasers = [i - df["Time"].tolist()[0] for i in nonlasers]
        nan_removed_area = np.delete(Area, np.argwhere(np.isnan(Area)))
        nan_removed_times = np.delete(Time, np.argwhere(np.isnan(Area)))
        areas_at_detection = []
        for dilation_time in Dil_times:
      
                ix = find_nearest(Time,dilation_time)
                areas_at_detection.append(Area[ix])
          
        print(mouse_id,date,np.min(areas_at_detection),np.mean(areas_at_detection),(len(Time)/Time[-1]))
     
        new_dict = { "Thresh" : np.min(areas_at_detection),"ID":mouse_id,"Date":date,"fps" : (len(Time)/Time[-1]),"Actual":np.mean(areas_at_detection),"Percent diff":(np.mean(areas_at_detection)- np.min(areas_at_detection))/(np.min(areas_at_detection))}
        df_list.append(new_dict)


validation_df = pd.DataFrame.from_dict(df_list)


In [ ]:
sns.lmplot(data = validation_df, x= "Percent diff", y = "fps")
pearsonr(validation_df["Percent diff"],validation_df["fps"])